<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/DL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

     |████████████████████████████████| 2.0MB 6.1MB/s 
     |████████████████████████████████| 133kB 23.4MB/s 
     |████████████████████████████████| 163kB 20.4MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=3cc26759b21a222ac22023974292d2bb8893c040d61090081156fbf6431340fc
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=6fe29cc09730e5a9628e031cc6b9ed19231a0970c31e78ecc87138c653471f49
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
import wandb
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#importing the fasihion mnist dataset from keras
from keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.metrics import accuracy_score, mean_squared_error
import tensorflow as tf

In [ ]:
def sigmoid(x):
    temp=np.zeros((len(x),1))
    for i in range(len(x)):
        if(x[i]>=0):
            temp[i]=1.0/(1.0+np.exp(-x[i]))
        else:
            temp[i]=np.exp(x[i])/(1.0+np.exp(x[i]))
    return temp

In [ ]:
def sigmoid_dif(x):
    return x*(1-x)

In [ ]:
def tanh(x):
    temp=np.zeros((len(x),1))
    for i in range(len(x)):
        temp[i]=np.exp(x[i])-np.exp(-x[i])/np.exp(x[i])+np.exp(-x[i])
    return temp

In [ ]:
def tanh_dif(x):
    return (1 - (x)**2)

In [ ]:


def relu(x):
    temp=np.zeros((len(x),1))
    for i in range(len(x)):
        if x[i]>0:
            temp[i]=x[i]
    return temp

In [ ]:

def relu_dif(x):
    temp=np.zeros((len(x),1))
    for i in range(len(x)):
        if x[i]>0:
            temp[i]=1
    return temp


In [ ]:
def softmax(x):
    mx=np.max(x)
    x=x-mx
    return np.exp(x)/np.sum(np.exp(x))

In [ ]:
def initialize_network(layer_size_list,mthd):
    weights={}
    bias={}
    if mthd=='random':
        for i,x,y in zip(range(len(layer_size_list)-1),layer_size_list[:-1],layer_size_list[1:]):
            weights[i+1]=np.random.randn(y,x)
        for i,x in zip(range(len(layer_size_list)-1),layer_size_list[1:]):
            bias[i+1]=np.zeros((x,1))
        return weights,bias
    elif mthd=='xavier':
        initializer = tf.keras.initializers.GlorotNormal()
        for i,x,y in zip(range(len(layer_size_list)-1),layer_size_list[:-1],layer_size_list[1:]):
            weights[i+1]=np.array(initializer(shape=(y, x)))
        for i,x in zip(range(len(layer_size_list)-1),layer_size_list[1:]):
            bias[i+1]=np.zeros((x,1))
        return weights,bias

In [ ]:
def forward_propagation(x,weights,bias,mthd):
    a={}
    h={}
    h[0]=x
    layers=len(weights)
    if mthd=='sigmoid':
        for i in range(layers-1):
            a[i+1]=np.matmul(weights[i+1],h[i])+bias[i+1]
            h[i+1]=sigmoid(a[i+1])
        a[layers]=np.matmul(weights[layers],h[layers-1])+bias[layers]
        h[layers]=softmax(a[layers])
        #return h[layers]
        return a,h
    elif mthd=='tanh':
        for i in range(layers-1):
            a[i+1]=np.matmul(weights[i+1],h[i])+bias[i+1]
            h[i+1]=tanh(a[i+1])
        a[layers]=np.matmul(weights[layers],h[layers-1])+bias[layers]
        h[layers]=softmax(a[layers])
        #return h[layers]
        return a,h
    elif mthd=='relu':
        for i in range(layers-1):
            a[i+1]=np.matmul(weights[i+1],h[i])+bias[i+1]
            h[i+1]=relu(a[i+1])
        a[layers]=np.matmul(weights[layers],h[layers-1])+bias[layers]
        h[layers]=softmax(a[layers])
        #return h[layers]
        return a,h
        

In [ ]:
def back_propagation(x,y,weights,bias,mthd):
    a,h=forward_propagation(x.reshape(784,1),weights,bias,mthd)
    dW={}
    dB={}
    dH={}
    dA={}
    layers=len(weights)
    y_label=[0]*10
    y_label[y]=1
    y_label=np.array(y_label).reshape(10,1)
    dA[layers]=h[layers].reshape(10,1)-y_label
    for i in range(layers,0,-1):
        dW[i]=np.matmul(dA[i],h[i-1].T)
        dB[i]=dA[i]
        dH[i-1]=np.matmul(weights[i].T,dA[i])
        if mthd=='sigmoid':
            dA[i-1]=np.multiply(dH[i-1],sigmoid_dif(h[i-1]))
        elif mthd=='tanh':
            dA[i-1]=np.multiply(dH[i-1],tanh_dif(h[i-1]))
        elif mthd=='relu':
            dA[i-1]=np.multiply(dH[i-1],relu_dif(h[i-1]))
    return dW,dB    

In [ ]:
#Optimization Functions

In [ ]:
def stochastic_gradient_descent(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0,batch_size=10):
    for e in range(epochs):
        dw={}
        db={}
        ce_train=0
        ce_val=0
        num_point_seen=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,weights,bias,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
            num_point_seen+=1
            if num_point_seen==batch_size:
                for z in range(len(layers_size_list)-1):
                    weights[z+1]=weights[z+1]-learning_rate*dw[z+1]/batch_size-learning_rate*alpha*weights[z+1]
                    bias[z+1]=bias[z+1]-learning_rate*db[z+1]/batch_size
                for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
                    dw[i+1]=np.zeros((y,x))
                for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
                    db[i+1]=np.zeros((x,1))
                num_point_seen=0
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
            #mse1+=mse(res[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias

In [ ]:

def momentum_gradient_descent(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0):
    for e in range(epochs):
        dw={}
        db={}
        prev_vw={}
        prev_vb={}
        ce_train=0
        ce_val=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
            prev_vw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
            prev_vb[i+1]=np.zeros((x,1))
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,weights,bias,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
        m=len(train_images[0:1000])
        v_w={}
        v_b={}
        gamma=0.9
        for i in range(len(layers_size_list)-1):
            v_w[i+1]=gamma*prev_vw[i+1]+learning_rate*dw[i+1]#/m
            v_b[i+1]=gamma*prev_vb[i+1]+learning_rate*db[i+1]#/m
            weights[i+1]=weights[i+1]-v_w[i+1]-learning_rate*alpha*weights[i+1] #this is l2 regularisation
            bias[i+1]=bias[i+1]-v_b[i+1]
            prev_vw[i+1]=v_w[i+1]
            prev_vb[i+1]=v_b[i+1]
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
            #mse1+=mse(res[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias





In [ ]:


def nestrov_gradient_descent(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0):
    for e in range(epochs):
        dw={}
        db={}
        prev_vw={}
        prev_vb={}
        ce_train=0
        ce_val=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
            prev_vw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
            prev_vb[i+1]=np.zeros((x,1))
        v_w={}
        v_b={}
        gamma=0.9
        for i in range(len(layers_size_list)-1):
            v_w[i+1]=gamma*prev_vw[i+1]
            v_b[i+1]=gamma*prev_vb[i+1]
        tempw={}
        tempb={}
        for i in range(len(layers_size_list)-1):
            tempw[i+1]=weights[i+1]-v_w[i+1]
            tempb[i+1]=bias[i+1]-v_b[i+1]
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,tempw,tempb,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
        m=len(train_images[0:1000])
        for i in range(len(layers_size_list)-1):
            v_w[i+1]=gamma*prev_vw[i+1]+learning_rate*dw[i+1]#/m
            v_b[i+1]=gamma*prev_vb[i+1]+learning_rate*db[i+1]#/m
            weights[i+1]=weights[i+1]-v_w[i+1]-learning_rate*alpha*weights[i+1]
            bias[i+1]=bias[i+1]-v_b[i+1]
            prev_vw[i+1]=v_w[i+1]
            prev_vb[i+1]=v_b[i+1]
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
            #mse1+=mse(res[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias


In [ ]:
def rmsprop(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0):
    res=[]
    vw={}
    vb={}
    for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
        vw[i+1]=np.zeros((y,x))
    for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
        vb[i+1]=np.zeros((x,1))
    eps=1e-8
    beta1=0.9
    for e in range(epochs):
        dw={}
        db={}
        ce_train=0
        ce_val=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,weights,bias,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
        #m=len(train_images[0:1000])
        for l in range(len(layers_size_list)-1):
            vw[l+1]=vw[l+1]*beta1+(1-beta1)*(dw[l+1]**2)
            vb[l+1]=vb[l+1]*beta1+(1-beta1)*(db[l+1]**2)
        for i in range(len(layers_size_list)-1):
            weights[i+1]=weights[i+1]-(learning_rate)*(dw[i+1]/np.sqrt(vw[i+1]+eps))-learning_rate*alpha*weights[i+1]
            bias[i+1]=bias[i+1]-(learning_rate)*(db[i+1]/np.sqrt(vb[i+1]+eps))
        
        #below code is for testing accuracy on train data
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
            #mse1+=mse(res[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias

In [ ]:
def adam(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0):
    res=[]
    vw={}
    vb={}
    mw={}
    mb={}
    mw_hat={}
    mb_hat={}
    vw_hat={}
    vb_hat={}
    for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
        vw[i+1]=np.zeros((y,x))
        mw[i+1]=np.zeros((y,x))
        mw_hat[i+1]=np.zeros((y,x))
        vw_hat[i+1]=np.zeros((y,x))
        
    for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
        vb[i+1]=np.zeros((x,1))
        mb[i+1]=np.zeros((x,1))
        mb_hat[i+1]=np.zeros((x,1))
        vb_hat[i+1]=np.zeros((x,1))

    eps=1e-8
    beta1=0.9
    beta2=0.999
    for e in range(epochs):
        dw={}
        db={}
        ce_train=0
        ce_val=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,weights,bias,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
        #m=len(train_images[0:1000])
        for l in range(len(layers_size_list)-1):
            mw[l+1]=mw[l+1]*beta1+(1-beta1)*(dw[l+1])
            mb[l+1]=mb[l+1]*beta1+(1-beta1)*(db[l+1])
            
        
        for l in range(len(layers_size_list)-1):
            vw[l+1]=vw[l+1]*beta2+(1-beta2)*(dw[l+1]**2)
            vb[l+1]=vb[l+1]*beta2+(1-beta2)*(db[l+1]**2)
            
        for l in range(len(layers_size_list)-1):
            mw_hat[l+1]=mw[l+1]/(1-(beta1**(e+1)))
            mb_hat[l+1]=mb[l+1]/(1-(beta1**(e+1)))
            
        for l in range(len(layers_size_list)-1):
            vw_hat[l+1]=vw[l+1]/(1-(beta2**(e+1)))
            vb_hat[l+1]=vb[l+1]/(1-(beta2**(e+1)))
            
        for i in range(len(layers_size_list)-1):
            weights[i+1]=weights[i+1]-(learning_rate)*(mw_hat[i+1]/np.sqrt(vw_hat[i+1]+eps))-learning_rate*alpha*weights[i+1]
            bias[i+1]=bias[i+1]-(learning_rate)*(mb_hat[i+1]/np.sqrt(vb_hat[i+1]+eps))
        
        #below code is for testing accuracy on train data
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias

In [ ]:
def nadam(weights,bias,epochs,layers_size_list,train_images,train_labels,learning_rate,mthd,alpha=0):
    res=[]
    vw={}
    vb={}
    mw={}
    mb={}
    mw_hat={}
    mb_hat={}
    vw_hat={}
    vb_hat={}
    for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
        vw[i+1]=np.zeros((y,x))
        mw[i+1]=np.zeros((y,x))
        mw_hat[i+1]=np.zeros((y,x))
        vw_hat[i+1]=np.zeros((y,x))
        
    for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
        vb[i+1]=np.zeros((x,1))
        mb[i+1]=np.zeros((x,1))
        mb_hat[i+1]=np.zeros((x,1))
        vb_hat[i+1]=np.zeros((x,1))

    eps=1e-8
    beta1=0.9
    beta2=0.999
    for e in range(epochs):
        dw={}
        db={}
        ce_train=0
        ce_val=0
        for i,x,y in zip(range(len(layers_size_list)-1),layers_size_list[:-1],layers_size_list[1:]):
            dw[i+1]=np.zeros((y,x))
        for i,x in zip(range(len(layers_size_list)-1),layers_size_list[1:]):
            db[i+1]=np.zeros((x,1))
        for j,k in zip(train_images[0:1000],train_labels[0:1000]):
            tdw,tdb=back_propagation(j,k,weights,bias,mthd)
            for l in range(len(layers_size_list)-1):
                dw[l+1]+=tdw[l+1]
                db[l+1]+=tdb[l+1]
        #m=len(train_images[0:1000])
        for l in range(len(layers_size_list)-1):
            mw[l+1]=mw[l+1]*beta1+(1-beta1)*(dw[l+1])
            mb[l+1]=mb[l+1]*beta1+(1-beta1)*(db[l+1])
            
        
        for l in range(len(layers_size_list)-1):
            vw[l+1]=vw[l+1]*beta2+(1-beta2)*(dw[l+1]**2)
            vb[l+1]=vb[l+1]*beta2+(1-beta2)*(db[l+1]**2)
            
        for l in range(len(layers_size_list)-1):
            mw_hat[l+1]=mw[l+1]/(1-(beta1**(e+1)))
            mb_hat[l+1]=mb[l+1]/(1-(beta1**(e+1)))
            
        for l in range(len(layers_size_list)-1):
            vw_hat[l+1]=vw[l+1]/(1-(beta2**(e+1)))
            vb_hat[l+1]=vb[l+1]/(1-(beta2**(e+1)))
        #xk = xk - (n/(vc_k**0.5 + epsilon))*(b1*mc_k + (1-b1)*gt/(1-b1**iter_count))  
        for i in range(len(layers_size_list)-1):
            numrw=beta1*mw_hat[i+1]+(1-beta1)/(1-beta1**(e+1))*dw[i+1]
            weights[i+1]=weights[i+1]-(learning_rate)*(numrw/np.sqrt(vw_hat[i+1]+eps))-learning_rate*alpha*weights[i+1]
            
            numrb=beta1*mb_hat[i+1]+(1-beta1)/(1-beta1**(e+1))*db[i+1]
            bias[i+1]=bias[i+1]-(learning_rate)*(numrb/np.sqrt(vb_hat[i+1]+eps))
        
        #below code is for testing accuracy on train data
        res=[]
        for i in train_images[0:1000]:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res.append(h[len(weights)])
        y_pred=[]
        for i in res:
            j=list(i)
            temp=j.index(max(j))
            y_pred.append(temp)
        train_acc=accuracy_score(y_pred, train_labels[0:1000])
        print(e+1,":",train_acc)
        #the below code is for calculating cross entropy
        for i in range(len(train_labels[0:1000])):
            j=[0]*10
            j[train_labels[i]]=1
            ce_train+=cross_entropy(res[i],j)
        res_test=[]
        for i in test_images:
            a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
            res_test.append(h[len(weights)])
        for i in range(len(test_labels)):
            j=[0]*10
            j[test_labels[i]]=1
            ce_val+=cross_entropy(res_test[i],j)
            #mse1+=mse(res[i],j)
        #print("CE-loss:",ce/len(train_labels[0:1000]))
        #print("MSE-loss:",mse1/len(train_labels[0:1000]))
        #if((e+1)%100==0):
        #print(e,":",train_acc)
        test_acc=test_accuracy(weights,bias,test_images,test_labels,mthd)
        wandb.log({
        "Epoch": e,
        "Train Loss": ce_train/len(train_labels[0:1000]),
        "Train Acc": train_acc,
        "Valid Loss": ce_val/len(test_labels),
        "Valid Acc": test_acc})
        #res.append(train_acc)
        #print(e,":",train_acc)
    return weights,bias

In [ ]:
def mse(pred,label):
    label=np.array(label).reshape(10,1)
    mse=0
    for i in range(len(label)):
        mse+=(pred[i]-label[i])**2
    return mse[0]

In [ ]:


def cross_entropy(pred,label):
    yl=np.multiply(pred,np.array(label).reshape(10,1))
    yl=yl[yl!=0]
    yl=-np.log(yl)
    #yl=np.mean(yl)
    return yl[0]

In [ ]:
def test_accuracy(weights,bias,test_images,test_labels,mthd):
    res=[]
    for i in test_images:
        a,h=forward_propagation(i.reshape(784,1),weights,bias,mthd)
        res.append(h[len(weights)])
    y_pred=[]
    for i in res:
        j=list(i)
        temp=j.index(max(j))
        y_pred.append(temp)
    test_acc=accuracy_score(y_pred, test_labels)
    return test_acc